In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

In [ ]:
# Reload the web page and execute this cell
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.8.16 (default, Mar  2 2023, 03:21:46) 
[GCC 11.2.0]


In [ ]:
pip install tensorflow

In [ ]:
pip install recommenders

In [ ]:
pip install tf_slim

In [ ]:
import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.datasets.python_splitters import *
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.8.16 (default, Mar  2 2023, 03:21:46) 
[GCC 11.2.0]
Pandas version: 1.2.3
Tensorflow version: 2.12.0


In [ ]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 20
BATCH_SIZE = 128

SEED = 42

In [ ]:
import pandas as pd
df= pd.read_csv("/content/full_data.csv")


In [ ]:
df = df.rename(columns={'userId': 'userID'})


In [ ]:
df = df.rename(columns={'movieId': 'itemID'})


In [ ]:
df["IMAX"].value_counts()

0    96691
1     4145
Name: IMAX, dtype: int64

In [ ]:
df['userID'].value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
320      20
569      20
442      20
257      20
406      20
Name: userID, Length: 610, dtype: int64

In [ ]:
train, test = python_stratified_split(df, ratio=0.75)

In [ ]:
train["userID"].unique()== test["userID"].unique()

In [ ]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [ ]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [ ]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

100%|██████████| 610/610 [00:55<00:00, 10.91it/s]


In [ ]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-06-24 19:11:03.613343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 19:11:04.141210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 19:11:04.141657: I tensorflow/compiler/

In [ ]:
model.fit(data)

In [ ]:
with Timer() as test_time:

  users, items, preds = [], [], []
  item = list(train.itemID.unique())
  for user in train.userID.unique():
      user = [user] * len(item)
      users.extend(user)
      items.extend(item)
      preds.extend(list(model.predict(user, item, is_list=True)))

  all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

  merged = pd.merge(train[["userID", "itemID","rating"]], all_predictions, on=["userID", "itemID"], how="outer")
  all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

In [ ]:
train[["userID", "itemID"]].head(5)

,userID,itemID
15656,1,3578
6579,1,1127
1135,1,157
12997,1,2617
1568,1,260


In [ ]:
all_predictions.head(5)

,userID,itemID,prediction
75630,1,122882,0.002459
75631,1,80906,0.003930
75632,1,74458,0.018611
75633,1,106782,0.008386
75634,1,58559,0.116817


In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.041111
NDCG:	0.218612
Precision@K:	0.204098
Recall@K:	0.084241


In [ ]:
model.predict(2, 91658)

0.9466220736503601

In [ ]:
all_predictions[all_predictions["userID"]==4].sort_values('prediction',ascending=False)

,userID,itemID,prediction
102030,4,111,9.922211e-01
103366,4,903,9.902501e-01
102007,4,1299,9.871714e-01
102003,4,904,9.848726e-01
101954,4,858,9.824764e-01
...,...,...,...
106908,4,5883,1.330203e-08
106985,4,6316,1.328889e-08
106993,4,2559,9.886341e-09
106929,4,48678,7.588930e-09


In [ ]:
all_predictions.to_csv('all_predictions.csv', index=False)

In [ ]:
df2= pd.read_csv("/content/all_predictions.csv")

In [ ]:
df2.head(5)

,userID,itemID,prediction
0,1,122882,0.002459
1,1,80906,0.003930
2,1,74458,0.018611
3,1,106782,0.008386
4,1,58559,0.116817


In [ ]:
df2[df2["userID"]==4].sort_values('prediction',ascending=False)

,userID,itemID,prediction
26400,4,111,9.922211e-01
27736,4,903,9.902501e-01
26377,4,1299,9.871714e-01
26373,4,904,9.848726e-01
26324,4,858,9.824764e-01
...,...,...,...
31278,4,5883,1.330203e-08
31355,4,6316,1.328889e-08
31363,4,2559,9.886341e-09
31299,4,48678,7.588930e-09


In [ ]:
model.predict(4,111)

0.9922210574150085